In [1]:
import pandas as pd
import jsonlines
import os
from glob import glob

In [2]:
files = glob("../outputs/*/*/prometheus-13b*/alpaca-eval.json")
dfs = []

def filter_score(x):
    if x["score"] > 0:
        return x["score"]
    if "overall score is" in x["critic"]:
        score = int(x["critic"].split("overall score is")[1].split(".")[0].strip().replace(",","").split(" ", 1)[0])
        return score
    elif "a score of " in x["critic"]:
        score = x["critic"].split("a score of ")[1].split(" ")[0].split(".")[0].split(",")[0].strip()
        if score:
            try:
                return int(score)
            except ValueError:
                print(score)
                return -1
        else:
            return -1
    else:
        return -1

for file in files:
    df = pd.read_json(file, lines=True)
    df["score"] = df.judge.apply(filter_score)
    dfs.append(df)

df = pd.concat(dfs)
df["output_len"] = df.output.apply(len)
df.shape

(14490, 8)

In [3]:
df[df.score > 0].groupby("generator").agg({
    "score": ["mean", "median", "std", "count"],
})

score                   \
                                                        mean median       std   
generator                                                                       
HuggingFaceH4/zephyr-7b-beta                        3.855901    5.0  1.490135   
alignment-handbook/zephyr-7b-dpo-full               4.305590    5.0  1.259207   
alignment-handbook/zephyr-7b-sft-full               2.879503    3.0  1.562668   
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...  3.734161    4.0  1.575272   
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...  3.166460    4.0  1.585732   
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...  2.944099    3.0  1.602425   
heegyu/0510-dco-v2-7b@lr2e-4-beta0.01-epoch-1       3.978882    5.0  1.513883   
heegyu/0510-dco-v4-7b@lr2e-4-beta0.01-epoch-1       4.078261    5.0  1.411603   
heegyu/0510-dpo-7b@lr2e-4-beta0.01-epoch-1          3.884472    5.0  1.545438   
heegyu/0510-dpo-7b@lr3e-4-beta0.01-epoch-1          3.524224    4.0  1.691645   
heegyu/0513-dco-7b@lr1e-4-beta0.01-epoch-1          3.459627    4.0  1.599883   
heegyu/0513-dco-7b@lr2e-4-beta0.01-epoch-1          3.408696    4.0  1.615588   
heegyu/0513-dco-v1d-7b@lr1e-4-beta0.01-epoch-1      3.347935    4.0  1.621934   
heegyu/0513-dco-v4-7b@lr1e-4-beta0.01-epoch-1       3.335821    4.0  1.623047   
heegyu/0513-dco-v4d-7b@lr1e-4-beta0.01-epoch-1      3.410959    4.0  1.586407   
heegyu/0513-dpo-7b@lr1e-4-beta0.01-epoch-1          3.524224    4.0  1.569603   
heegyu/0513-dpo-7b@lr2e-4-beta0.01-epoch-1          3.614907    4.0  1.623299   
heegyu/0513-dpo-7b@lr3e-4-beta0.01-epoch-1          3.443478    4.0  1.672383   

                                                          
                                                   count  
generator                                                 
HuggingFaceH4/zephyr-7b-beta                         805  
alignment-handbook/zephyr-7b-dpo-full                805  
alignment-handbook/zephyr-7b-sft-full                805  
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...   805  
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...   805  
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...   805  
heegyu/0510-dco-v2-7b@lr2e-4-beta0.01-epoch-1        805  
heegyu/0510-dco-v4-7b@lr2e-4-beta0.01-epoch-1        805  
heegyu/0510-dpo-7b@lr2e-4-beta0.01-epoch-1           805  
heegyu/0510-dpo-7b@lr3e-4-beta0.01-epoch-1           805  
heegyu/0513-dco-7b@lr1e-4-beta0.01-epoch-1           805  
heegyu/0513-dco-7b@lr2e-4-beta0.01-epoch-1           805  
heegyu/0513-dco-v1d-7b@lr1e-4-beta0.01-epoch-1       799  
heegyu/0513-dco-v4-7b@lr1e-4-beta0.01-epoch-1        804  
heegyu/0513-dco-v4d-7b@lr1e-4-beta0.01-epoch-1       803  
heegyu/0513-dpo-7b@lr1e-4-beta0.01-epoch-1           805  
heegyu/0513-dpo-7b@lr2e-4-beta0.01-epoch-1           805  
heegyu/0513-dpo-7b@lr3e-4-beta0.01-epoch-1           805

In [19]:
df[df.score > 0].groupby("generator").agg({
    "score": "mean",
    "output_len": ["mean", "median", "std", "count"],
})

score   output_len  \
                                                        mean         mean   
generator                                                                   
HuggingFaceH4/zephyr-7b-beta                        3.418206  1402.920844   
alignment-handbook/zephyr-7b-dpo-full               3.612434  1500.129630   
alignment-handbook/zephyr-7b-sft-full               3.133247   843.956016   
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...  3.367725  1445.511905   
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...  3.265252  1178.567639   
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...  3.106632   936.213264   
heegyu/0510-dco-v2-7b@lr2e-4-beta0.01-epoch-1       3.315160  1547.502660   
heegyu/0510-dco-v4-7b@lr2e-4-beta0.01-epoch-1       3.492696  1294.823373   
heegyu/0510-dpo-7b@lr2e-4-beta0.01-epoch-1          3.312583  1433.533775   
heegyu/0510-dpo-7b@lr3e-4-beta0.01-epoch-1          3.107330  1458.083770   
heegyu/0513-dco-7b@lr1e-4-beta0.01-epoch-1          3.457067  1080.922061   
heegyu/0513-dco-v1d-7b@lr1e-4-beta0.01-epoch-1      3.303030  1182.076416   
heegyu/0513-dco-v4-7b@lr1e-4-beta0.01-epoch-1       3.398438  1150.727865   
heegyu/0513-dco-v4d-7b@lr1e-4-beta0.01-epoch-1      3.430789  1078.680466   
heegyu/0513-dpo-7b@lr1e-4-beta0.01-epoch-1          3.407552  1033.217448   
heegyu/0513-dpo-7b@lr2e-4-beta0.01-epoch-1          3.495472   967.239327   
heegyu/0513-dpo-7b@lr3e-4-beta0.01-epoch-1          3.363395  1071.137931   

                                                                               
                                                    median          std count  
generator                                                                      
HuggingFaceH4/zephyr-7b-beta                        1333.0  1051.696655   758  
alignment-handbook/zephyr-7b-dpo-full               1411.0  1309.046160   756  
alignment-handbook/zephyr-7b-sft-full                765.0   726.533005   773  
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...  1437.5   860.763937   756  
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...  1141.0   713.914145   754  
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...   869.0   742.425422   769  
heegyu/0510-dco-v2-7b@lr2e-4-beta0.01-epoch-1       1526.0   842.936287   752  
heegyu/0510-dco-v4-7b@lr2e-4-beta0.01-epoch-1       1272.0   751.236411   753  
heegyu/0510-dpo-7b@lr2e-4-beta0.01-epoch-1          1417.0   773.745662   755  
heegyu/0510-dpo-7b@lr3e-4-beta0.01-epoch-1          1431.0  1019.599961   764  
heegyu/0513-dco-7b@lr1e-4-beta0.01-epoch-1          1009.0  1185.856958   757  
heegyu/0513-dco-v1d-7b@lr1e-4-beta0.01-epoch-1      1008.0  1321.205402   759  
heegyu/0513-dco-v4-7b@lr1e-4-beta0.01-epoch-1       1082.0   979.280988   768  
heegyu/0513-dco-v4d-7b@lr1e-4-beta0.01-epoch-1       942.0  1192.350033   773  
heegyu/0513-dpo-7b@lr1e-4-beta0.01-epoch-1           968.5  1053.053954   768  
heegyu/0513-dpo-7b@lr2e-4-beta0.01-epoch-1           790.0  1196.458556   773  
heegyu/0513-dpo-7b@lr3e-4-beta0.01-epoch-1           872.0  1242.085542   754

# PairRM

In [3]:
files = glob("../outputs/*/*/pairrm/*.json")
dfs = []

for file in files:
    df = pd.read_json(file, lines=True)
    df["dataset"] = os.path.basename(file).split(".")[0]
    dfs.append(df)

df = pd.concat(dfs)
df.shape

(9780, 6)

In [11]:
df.groupby(["dataset", "generator"]).agg({
    "judge": ["mean", "median", "std", "count"],
})

TypeError: agg function failed [how->mean,dtype->object]